In [1]:
import pandas as pd
import os
import numpy as np
import dicom
import re
import xlsxwriter
import datetime
from datetime import datetime

In [4]:
#path = "R:\\groups\\valcour\\kevin\\test_files_for_scripts\\BK_tracking_test"
path="C:\\Users\\sf287610\\Documents\\01 SEARCH\\10 SEARCH018 Analysis\\MRS"
xlx_file = "Pom_BG-cleaned_2018-07-09.xlsx"
xlx_sheet = ["pre-proc",1]

os.chdir(path)
print "done"

done


In [6]:
MRS_in = pd.read_excel(xlx_file,sheet_name=xlx_sheet[0])

#if len(xlx_sheet) > 1:
#    for i in range(1, len(xlx_sheet)):
#        MRS_in_entry = pd.read_excel(xlx_file,sheet_name=i)
#        MRS_in = MRS_in.append(MRS_in_entry)
        
    



print MRS_in

    S018 ID  S018 Visit  S010 ID   QC      QC Comments     Ala   Ala %SD  \
0         1           0      237  NaN              NaN  4.8090      51.0   
1         1          12      237  NaN              NaN  0.0000     999.0   
2         1          18      237  NaN              NaN  5.5610      44.0   
3         3           0      240  NaN              NaN  0.0000     999.0   
4         3          12      240  NaN              NaN  0.0000     999.0   
5         3          18      240  NaN              NaN  0.9190     160.0   
6         4           0      241  NaN              NaN  1.3970     205.0   
7         4          12      241  NaN              NaN  0.0000     999.0   
8         4          18      241  NaN              NaN  0.6380     423.0   
9         6           0      249  NaN              NaN  0.0000     999.0   
10        6          12      249  NaN              NaN  0.0000     999.0   
11        6          18      249  NaN              NaN  2.2630      71.0   
12        7 

In [371]:
len(xlx_sheet) > 1

True

In [17]:
MRS_QA = MRS_in
#MRS_QA = pd.DataFrame(columns=["Study","PIDN","Visit"]+MRS_in.columns.tolist())
#MRS_QA_entry = pd.DataFrame


# importing the spreadsheet for QA analysis
for i in range(len(MRS_in)):
    #MRS_QA_entry = MRS_in.loc[i]
    if not pd.isnull(MRS_in.loc[i,MRS_in.columns[0]]):
        #validation_str = MRS_QA_entry[MRS_in.columns[0]].split("_")
        #Study = validation_str[0][:-4]
        Study = "S018"
        #PIDN = validation_str[0][-4:]
        PIDN = MRS_in.loc[i,"S018 ID"]
        #Visit = validation_str[1]
        Visit = MRS_in.loc[i,"S018 Visit"]
        #Location = validation_str[2]
        MRS_QA.loc[i,"Study"] = Study
        MRS_QA.loc[i,"PIDN"] = PIDN
        MRS_QA.loc[i,"Visit"] = Visit
        #MRS_QA_entry.at["Location"] = Location
        
        
        # converts full_ID to Study, PIDN, Visit, Location
        #MRS_QA = MRS_QA.append( MRS_QA_entry, ignore_index=True).drop(columns=[MRS_in.columns[0]])
print "Done"
print MRS_QA

Done
    S018 ID  S018 Visit  S010 ID   QC      QC Comments     Ala   Ala %SD  \
0         1           0      237  NaN              NaN  4.8090      51.0   
1         1          12      237  NaN              NaN  0.0000     999.0   
2         1          18      237  NaN              NaN  5.5610      44.0   
3         3           0      240  NaN              NaN  0.0000     999.0   
4         3          12      240  NaN              NaN  0.0000     999.0   
5         3          18      240  NaN              NaN  0.9190     160.0   
6         4           0      241  NaN              NaN  1.3970     205.0   
7         4          12      241  NaN              NaN  0.0000     999.0   
8         4          18      241  NaN              NaN  0.6380     423.0   
9         6           0      249  NaN              NaN  0.0000     999.0   
10        6          12      249  NaN              NaN  0.0000     999.0   
11        6          18      249  NaN              NaN  2.2630      71.0   
12     

In [20]:
# Assuming " Ala" is first element of metabolites, change if necessary
mtb_start = MRS_QA.columns.get_loc(" Ala")
mtb_list = (MRS_QA.columns[mtb_start::3]).tolist()
mtb_sd = (MRS_QA.columns[(mtb_start+1)::3]).tolist()
mtb_cre_ratio = (MRS_QA.columns[(mtb_start+2)::3]).tolist()

print mtb_list

[u' Ala', u' Asp', u' Cr', u' PCr', u' GABA', u' Glc', u' Gln', u' Glu', u' GPC', u' PCh', u' GSH', u' Ins', u' Lac', u' NAA', u' NAAG', u' Scyllo', u' Tau', u' -CrCH2', u' GPC+PCh', u' NAA+NAAG', u' Cr+PCr', u' Glu+Gln', u' Lip13a', u' Lip13b', u' Lip09', u' MM09', u' Lip20', u' MM20', u' MM12', u' MM14', u' MM17', u' Lip13a+Lip13b', u' MM14+Lip13a+L', u' MM09+Lip09', u' MM20+Lip20', 'Study']


In [24]:
num_pts = len(MRS_QA.index)
num_mtbs = (len(MRS_QA.columns[mtb_start:]))/3
print num_pts,"Subject timepoints, " ,num_mtbs, "metabolites"

for i in MRS_QA.index:
    #print MRS_QA.loc[i,"PIDN"]
    for val,sd,rat in zip(mtb_list,mtb_sd,mtb_cre_ratio):
        print val, sd, rat
        if MRS_QA.loc[i,sd] > 25 or np.isnan(MRS_QA.loc[i,sd]):
            print "sd too high"
            MRS_QA.at[i,val]=np.nan
            MRS_QA.at[i,sd]=np.nan
            MRS_QA.at[i,rat]=np.nan

print "done"

MRS_out = MRS_QA


51 Subject timepoints,  36 metabolites
 Ala  Ala %SD  Ala/Cr+PCr
sd too high
 Asp  Asp %SD  Asp/Cr+PCr
sd too high
 Cr  Cr %SD  Cr/Cr+PCr
sd too high
 PCr  PCr %SD  PCr/Cr+PCr
sd too high
 GABA  GABA %SD  GABA/Cr+PCr
sd too high
 Glc  Glc %SD  Glc/Cr+PCr
sd too high
 Gln  Gln %SD  Gln/Cr+PCr
sd too high
 Glu  Glu %SD  Glu/Cr+PCr
 GPC  GPC %SD  GPC/Cr+PCr
 PCh  PCh %SD  PCh/Cr+PCr
sd too high
 GSH  GSH %SD  GSH/Cr+PCr
 Ins  Ins %SD  Ins/Cr+PCr
sd too high
 Lac  Lac %SD  Lac/Cr+PCr
sd too high
 NAA  NAA %SD  NAA/Cr+PCr
 NAAG  NAAG %SD  NAAG/Cr+PCr
sd too high
 Scyllo  Scyllo %SD  Scyllo/Cr+PCr
 Tau  Tau %SD  Tau/Cr+PCr
sd too high
 -CrCH2  -CrCH2 %SD  -CrCH2/Cr+PCr
 GPC+PCh  GPC+PCh %SD  GPC+PCh/Cr+PCr
 NAA+NAAG  NAA+NAAG %SD  NAA+NAAG/Cr+PCr
 Cr+PCr  Cr+PCr %SD  Cr+PCr/Cr+PCr
 Glu+Gln  Glu+Gln %SD  Glu+Gln/Cr+PCr
 Lip13a  Lip13a %SD  Lip13a/Cr+PCr
sd too high
 Lip13b  Lip13b %SD  Lip13b/Cr+PCr
sd too high
 Lip09  Lip09 %SD  Lip09/Cr+PCr
sd too high
 MM09  MM09 %SD  MM09/Cr+PCr
 Lip20  L

sd too high
 GABA  GABA %SD  GABA/Cr+PCr
sd too high
 Glc  Glc %SD  Glc/Cr+PCr
sd too high
 Gln  Gln %SD  Gln/Cr+PCr
sd too high
 Glu  Glu %SD  Glu/Cr+PCr
 GPC  GPC %SD  GPC/Cr+PCr
sd too high
 PCh  PCh %SD  PCh/Cr+PCr
sd too high
 GSH  GSH %SD  GSH/Cr+PCr
 Ins  Ins %SD  Ins/Cr+PCr
 Lac  Lac %SD  Lac/Cr+PCr
sd too high
 NAA  NAA %SD  NAA/Cr+PCr
 NAAG  NAAG %SD  NAAG/Cr+PCr
sd too high
 Scyllo  Scyllo %SD  Scyllo/Cr+PCr
sd too high
 Tau  Tau %SD  Tau/Cr+PCr
sd too high
 -CrCH2  -CrCH2 %SD  -CrCH2/Cr+PCr
sd too high
 GPC+PCh  GPC+PCh %SD  GPC+PCh/Cr+PCr
 NAA+NAAG  NAA+NAAG %SD  NAA+NAAG/Cr+PCr
 Cr+PCr  Cr+PCr %SD  Cr+PCr/Cr+PCr
 Glu+Gln  Glu+Gln %SD  Glu+Gln/Cr+PCr
 Lip13a  Lip13a %SD  Lip13a/Cr+PCr
sd too high
 Lip13b  Lip13b %SD  Lip13b/Cr+PCr
sd too high
 Lip09  Lip09 %SD  Lip09/Cr+PCr
sd too high
 MM09  MM09 %SD  MM09/Cr+PCr
 Lip20  Lip20 %SD  Lip20/Cr+PCr
sd too high
 MM20  MM20 %SD  MM20/Cr+PCr
 MM12  MM12 %SD  MM12/Cr+PCr
sd too high
 MM14  MM14 %SD  MM14/Cr+PCr
 MM17  MM17 %SD  M

 PCh  PCh %SD  PCh/Cr+PCr
 GSH  GSH %SD  GSH/Cr+PCr
 Ins  Ins %SD  Ins/Cr+PCr
 Lac  Lac %SD  Lac/Cr+PCr
sd too high
 NAA  NAA %SD  NAA/Cr+PCr
 NAAG  NAAG %SD  NAAG/Cr+PCr
sd too high
 Scyllo  Scyllo %SD  Scyllo/Cr+PCr
sd too high
 Tau  Tau %SD  Tau/Cr+PCr
sd too high
 -CrCH2  -CrCH2 %SD  -CrCH2/Cr+PCr
sd too high
 GPC+PCh  GPC+PCh %SD  GPC+PCh/Cr+PCr
 NAA+NAAG  NAA+NAAG %SD  NAA+NAAG/Cr+PCr
 Cr+PCr  Cr+PCr %SD  Cr+PCr/Cr+PCr
 Glu+Gln  Glu+Gln %SD  Glu+Gln/Cr+PCr
 Lip13a  Lip13a %SD  Lip13a/Cr+PCr
sd too high
 Lip13b  Lip13b %SD  Lip13b/Cr+PCr
sd too high
 Lip09  Lip09 %SD  Lip09/Cr+PCr
sd too high
 MM09  MM09 %SD  MM09/Cr+PCr
 Lip20  Lip20 %SD  Lip20/Cr+PCr
sd too high
 MM20  MM20 %SD  MM20/Cr+PCr
 MM12  MM12 %SD  MM12/Cr+PCr
sd too high
 MM14  MM14 %SD  MM14/Cr+PCr
sd too high
 MM17  MM17 %SD  MM17/Cr+PCr
sd too high
 Lip13a+Lip13b  Lip13a+Lip13b %SD  Lip13a+Lip13b/Cr+PCr
sd too high
 MM14+Lip13a+L  MM14+Lip13a+L %SD  MM14+Lip13a+L/Cr+PCr
sd too high
 MM09+Lip09  MM09+Lip09 %SD  MM09+

sd too high
 Asp  Asp %SD  Asp/Cr+PCr
sd too high
 Cr  Cr %SD  Cr/Cr+PCr
 PCr  PCr %SD  PCr/Cr+PCr
sd too high
 GABA  GABA %SD  GABA/Cr+PCr
sd too high
 Glc  Glc %SD  Glc/Cr+PCr
sd too high
 Gln  Gln %SD  Gln/Cr+PCr
 Glu  Glu %SD  Glu/Cr+PCr
 GPC  GPC %SD  GPC/Cr+PCr
sd too high
 PCh  PCh %SD  PCh/Cr+PCr
sd too high
 GSH  GSH %SD  GSH/Cr+PCr
sd too high
 Ins  Ins %SD  Ins/Cr+PCr
 Lac  Lac %SD  Lac/Cr+PCr
sd too high
 NAA  NAA %SD  NAA/Cr+PCr
 NAAG  NAAG %SD  NAAG/Cr+PCr
sd too high
 Scyllo  Scyllo %SD  Scyllo/Cr+PCr
sd too high
 Tau  Tau %SD  Tau/Cr+PCr
 -CrCH2  -CrCH2 %SD  -CrCH2/Cr+PCr
sd too high
 GPC+PCh  GPC+PCh %SD  GPC+PCh/Cr+PCr
 NAA+NAAG  NAA+NAAG %SD  NAA+NAAG/Cr+PCr
 Cr+PCr  Cr+PCr %SD  Cr+PCr/Cr+PCr
 Glu+Gln  Glu+Gln %SD  Glu+Gln/Cr+PCr
 Lip13a  Lip13a %SD  Lip13a/Cr+PCr
sd too high
 Lip13b  Lip13b %SD  Lip13b/Cr+PCr
sd too high
 Lip09  Lip09 %SD  Lip09/Cr+PCr
sd too high
 MM09  MM09 %SD  MM09/Cr+PCr
sd too high
 Lip20  Lip20 %SD  Lip20/Cr+PCr
sd too high
 MM20  MM20 %SD  M

 Glu  Glu %SD  Glu/Cr+PCr
sd too high
 GPC  GPC %SD  GPC/Cr+PCr
sd too high
 PCh  PCh %SD  PCh/Cr+PCr
sd too high
 GSH  GSH %SD  GSH/Cr+PCr
sd too high
 Ins  Ins %SD  Ins/Cr+PCr
sd too high
 Lac  Lac %SD  Lac/Cr+PCr
sd too high
 NAA  NAA %SD  NAA/Cr+PCr
sd too high
 NAAG  NAAG %SD  NAAG/Cr+PCr
sd too high
 Scyllo  Scyllo %SD  Scyllo/Cr+PCr
sd too high
 Tau  Tau %SD  Tau/Cr+PCr
sd too high
 -CrCH2  -CrCH2 %SD  -CrCH2/Cr+PCr
sd too high
 GPC+PCh  GPC+PCh %SD  GPC+PCh/Cr+PCr
sd too high
 NAA+NAAG  NAA+NAAG %SD  NAA+NAAG/Cr+PCr
sd too high
 Cr+PCr  Cr+PCr %SD  Cr+PCr/Cr+PCr
sd too high
 Glu+Gln  Glu+Gln %SD  Glu+Gln/Cr+PCr
sd too high
 Lip13a  Lip13a %SD  Lip13a/Cr+PCr
sd too high
 Lip13b  Lip13b %SD  Lip13b/Cr+PCr
sd too high
 Lip09  Lip09 %SD  Lip09/Cr+PCr
sd too high
 MM09  MM09 %SD  MM09/Cr+PCr
sd too high
 Lip20  Lip20 %SD  Lip20/Cr+PCr
sd too high
 MM20  MM20 %SD  MM20/Cr+PCr
sd too high
 MM12  MM12 %SD  MM12/Cr+PCr
sd too high
 MM14  MM14 %SD  MM14/Cr+PCr
sd too high
 MM17  MM17 %SD

Study PIDN Visit
 Ala  Ala %SD  Ala/Cr+PCr
sd too high
 Asp  Asp %SD  Asp/Cr+PCr
sd too high
 Cr  Cr %SD  Cr/Cr+PCr
sd too high
 PCr  PCr %SD  PCr/Cr+PCr
sd too high
 GABA  GABA %SD  GABA/Cr+PCr
sd too high
 Glc  Glc %SD  Glc/Cr+PCr
sd too high
 Gln  Gln %SD  Gln/Cr+PCr
 Glu  Glu %SD  Glu/Cr+PCr
 GPC  GPC %SD  GPC/Cr+PCr
sd too high
 PCh  PCh %SD  PCh/Cr+PCr
 GSH  GSH %SD  GSH/Cr+PCr
sd too high
 Ins  Ins %SD  Ins/Cr+PCr
 Lac  Lac %SD  Lac/Cr+PCr
sd too high
 NAA  NAA %SD  NAA/Cr+PCr
 NAAG  NAAG %SD  NAAG/Cr+PCr
sd too high
 Scyllo  Scyllo %SD  Scyllo/Cr+PCr
sd too high
 Tau  Tau %SD  Tau/Cr+PCr
sd too high
 -CrCH2  -CrCH2 %SD  -CrCH2/Cr+PCr
sd too high
 GPC+PCh  GPC+PCh %SD  GPC+PCh/Cr+PCr
 NAA+NAAG  NAA+NAAG %SD  NAA+NAAG/Cr+PCr
 Cr+PCr  Cr+PCr %SD  Cr+PCr/Cr+PCr
 Glu+Gln  Glu+Gln %SD  Glu+Gln/Cr+PCr
 Lip13a  Lip13a %SD  Lip13a/Cr+PCr
sd too high
 Lip13b  Lip13b %SD  Lip13b/Cr+PCr
sd too high
 Lip09  Lip09 %SD  Lip09/Cr+PCr
sd too high
 MM09  MM09 %SD  MM09/Cr+PCr
 Lip20  Lip20 %SD  

In [25]:
print MRS_QA

    S018 ID  S018 Visit  S010 ID   QC      QC Comments   Ala   Ala %SD  \
0         1           0      237  NaN              NaN   NaN       NaN   
1         1          12      237  NaN              NaN   NaN       NaN   
2         1          18      237  NaN              NaN   NaN       NaN   
3         3           0      240  NaN              NaN   NaN       NaN   
4         3          12      240  NaN              NaN   NaN       NaN   
5         3          18      240  NaN              NaN   NaN       NaN   
6         4           0      241  NaN              NaN   NaN       NaN   
7         4          12      241  NaN              NaN   NaN       NaN   
8         4          18      241  NaN              NaN   NaN       NaN   
9         6           0      249  NaN              NaN   NaN       NaN   
10        6          12      249  NaN              NaN   NaN       NaN   
11        6          18      249  NaN              NaN   NaN       NaN   
12        7           0      251  NaN 

In [317]:
#############################
# Calculate stats
#############################

for i in mtb_list:
    mtbsum = sum(np.isnan(MRS_QA[i])==False)
    MRS_out.at["count",i] = mtbsum

    #mtbmean = np.mean( MRS_QA[np.isnan(MRS_QA[i])==False][i])
    #MRS_out.at["average",i] = mtbmean
    
print MRS_out


      Study  PIDN Visit Location  month   Ala   Ala %SD   Ala/Cre   Asp  \
0      S010  0302    M0       BG    0.0   NaN       NaN       NaN   NaN   
1      S010  0304    M0       BG    0.0   NaN       NaN       NaN   NaN   
2      S010  0307    M0       BG    0.0   NaN       NaN       NaN   NaN   
3      S010  0316    M0       BG    0.0   NaN       NaN       NaN   NaN   
4      S010  0318    M0       BG    0.0   NaN       NaN       NaN   NaN   
5      S010  0320    M0       BG    0.0   NaN       NaN       NaN   NaN   
6      S010  0332    M0       BG    0.0   NaN       NaN       NaN   NaN   
7      S010  0339    M0       BG    0.0   NaN       NaN       NaN   NaN   
8      S010  0341    M0       BG    0.0   NaN       NaN       NaN   NaN   
9      S010  0343    M0       BG    0.0   NaN       NaN       NaN   NaN   
10     S010  0346    M0       BG    0.0   NaN       NaN       NaN   NaN   
11     S010  0347    M0       BG    0.0   NaN       NaN       NaN   NaN   
12     S010  0348    M0  

In [26]:
MRS_out

,S018 ID,S018 Visit,S010 ID,QC,QC Comments,Ala,Ala %SD,Ala/Cr+PCr,Asp,Asp %SD,...,MM14+Lip13a+L/Cr+PCr,MM09+Lip09,MM09+Lip09 %SD,MM09+Lip09/Cr+PCr,MM20+Lip20,MM20+Lip20 %SD,MM20+Lip20/Cr+PCr,Study,PIDN,Visit
0,1,0,237,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.048,15.130,11.0,1.252,NaN,NaN,NaN,S018,1.0,0.0
1,1,12,237,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.128,11.598,13.0,1.056,NaN,NaN,NaN,S018,1.0,12.0
2,1,18,237,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.475,9.870,13.0,0.787,NaN,NaN,NaN,S018,1.0,18.0
3,3,0,240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.485,14.442,11.0,1.123,NaN,NaN,NaN,S018,3.0,0.0
4,3,12,240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.439,12.764,11.0,1.031,NaN,NaN,NaN,S018,3.0,12.0
5,3,18,240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.287,16.734,12.0,1.508,NaN,NaN,NaN,S018,3.0,18.0
6,4,0,241,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.346,12.373,14.0,1.005,NaN,NaN,NaN,S018,4.0,0.0
7,4,12,241,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.370,11.571,11.0,0.967,NaN,NaN,NaN,S018,4.0,12.0
8,4,18,241,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,10.013,13.0,0.821,NaN,NaN,NaN,S018,4.0,18.0
9,6,0,249,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.050,12.784,9.0,0.994,NaN,NaN,NaN,S018,6.0,0.0


In [30]:
now = datetime.now()
date = now.strftime("%Y%m%d")

writer = pd.ExcelWriter("MRS_from_Pom_%s.xlsx" % date, engine = 'xlsxwriter')
MRS_out.to_excel(writer, index = False)
workbook = writer.book
worksheet = writer.sheets["%s" % "Sheet1"]

formatred = workbook.add_format({'bg_color':'#FFC7CE', 'font_color': '#9C0006'})

formatyellow = workbook.add_format({'bg_color':'#FFF99B'})

for i in range(mtb_start,3*len(mtb_list),3):
    
    if  1:
    
        print MRS_out.columns[i]
        #worksheet.conditional_format(
        #    1,i+1,num_pts,i+1,{"type": "average", "criteria": "2 std dev above", "format": formatred})
        #worksheet.conditional_format(
        #    1,i+1,num_pts,i+1,{"type": "average", "criteria": "2 std dev below", "format": formatred})
#worksheet.conditional_format(
#    1,mtb_start,num_pts,num_mtbs*3,{"type": "average", "criteria": "2 std dev above", "format": formatred})

writer.save()


 Ala
 Asp
 Cr
 PCr
 GABA
 Glc
 Gln
 Glu
 GPC
 PCh
 GSH
 Ins
 Lac
 NAA
 NAAG
 Scyllo
 Tau
 -CrCH2
 GPC+PCh
 NAA+NAAG
 Cr+PCr
 Glu+Gln
 Lip13a
 Lip13b
 Lip09
 MM09
 Lip20
 MM20
 MM12
 MM14
 MM17
 Lip13a+Lip13b
 MM14+Lip13a+L
 MM09+Lip09
 MM20+Lip20


In [301]:
for i in range(mtb_start,3*len(mtb_list),3):
    print MRS_out.columns[i]
    worksheet.conditional_format(
        1,i,num_pts,i,{"type": "average", "criteria": "2 std dev above", "format": formatred})

#range(mtb_start,3*len(mtb_list),3)

 Ala
 Asp
 Cre
 GABA
 Glc
 Gln
 Glu
 GPC
 PCh
 Lac
 mI
 NAA
 NAAG
 Scyllo
 Tau
 -CrCH2
 Gua
 GPC+PCh
 NAA+NAAG
 Glu+Gln
 Lip13a
 Lip13b
 Lip09
 MM09
 Lip20
 MM20
 MM12
 MM14
 MM17
 Lip13a+Lip13b
 MM14+Lip13a+L
 MM09+Lip09
